<a href="https://colab.research.google.com/github/JF11579/Adelphi/blob/main/Colab_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#How did we select our Models?

We chose LAssobecasue it is reccommended  for situtationsa where there are more Features than Observations.  We used Grid Serach on Lasso to optimize it sparameters.

Lasso's is a linear regression technique that methodically reduces the Feature with the lowest coeficiaent then it re-runs the regression and again eliminstes the lowest.  

The combinations are vast.  And what it we reduce each variable but a differnt value.  Thisis what Grid Search does

We also went with ..............

We used Lasso with Grid Search to select the Features

'Seasonal', 'Gray_cement_dispatch_comercial', 'ISE',
       'Construction_lic_area/unt_SIlag_4', 'IEClag_5', 'ISElag_6',
       'Gray_cement_dispatch_Factorylag_9', 'ICElag_9',
       'Housing_total_sales', 'Housing_total_initiation_SIlag_24',
       'Ordinary_creditlag_4', 'Ordinary_creditlag_6', 'working_day']

Here is the abridged version of the code

First time through we use LAsso and Grid Search to identify our Features

## Libraries

In [28]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
import plotly.express as px 
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso


## Data

In [59]:
data= pd.read_csv('/content/drive/MyDrive/Porcelana_OP_adelphi_2021_SCALED.csv')
data.head(2)

,date,Quantity,Seasonal,Consumer_loans,Construction_lic_unt,Construction_lic_area,Construction_lic_unt_SI,Construction_lic_unt_NO_SI,Construction_lic_area_SI,Construction_lic_area_NO_SI,Gray_cement_prod,Gray_cement_dispatch,Gray_cement_dispatch_Factory,Gray_cement_dispatch_comercial,Gray_cement_dispatch_contractor,Gray_cement_dispatch_other,DTF(interest_rate),GDP_Constr,GDP_Total,Inflation,USD_exchange,Oil-brent,Oil_WTI,ISE,ICC,IEC,ICE,Unemployment%,RADAR_Constr,RADAR_Rev,RADAR_Toilets,RADAR_HomeAppliance,RADAR_Paint,RADAR_Furniture,RADAR_Tools,RADAR_Elect,RADAR_Plumbing,RADAR_Wood,RADAR_Supply,RADAR_Remodel,...,Preferencial_creditlag_8,loan_overdraftslag_8,Credit_cardlag_8,Treasury_loanlag_8,Housing_loanlag_8,Consumer_creditlag_9,Consumer_microcreditlag_9,Ordinary_creditlag_9,Preferencial_creditlag_9,loan_overdraftslag_9,Credit_cardlag_9,Treasury_loanlag_9,Housing_loanlag_9,Consumer_creditlag_10,Consumer_microcreditlag_10,Ordinary_creditlag_10,Preferencial_creditlag_10,loan_overdraftslag_10,Credit_cardlag_10,Treasury_loanlag_10,Housing_loanlag_10,Consumer_creditlag_11,Consumer_microcreditlag_11,Ordinary_creditlag_11,Preferencial_creditlag_11,loan_overdraftslag_11,Credit_cardlag_11,Treasury_loanlag_11,Housing_loanlag_11,Consumer_creditlag_12,Consumer_microcreditlag_12,Ordinary_creditlag_12,Preferencial_creditlag_12,loan_overdraftslag_12,Credit_cardlag_12,Treasury_loanlag_12,Housing_loanlag_12,holiday,sunday,working_day
0,2017-01-01,1.503548,0.699923,112579.4297,11303.0,1185131.0,3587.0,7716.0,229478.0,955653.0,897063.518,913190.407,194276.020,502084.8330,171559.9665,45269.5875,0.0694,14753.34784,206242.568,5.47,2944.65,52.19,52.81,102.548972,-30.2,-28.3,-33.1,11.733267,1.044525e+11,1.323489e+11,1.727682e+11,1.050914e+11,8.847567e+10,8.354725e+10,5.008444e+10,9.372000e+10,3.037115e+10,3.136015e+10,1.585430e+11,1.050763e+12,...,4445890.861,6402426.269,3390803.016,550883.0739,1302564.560,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.00,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,3814229.013,632387.5921,1627959.289,4068020.900,438682.2679,6513619.182,6137144.374,6675101.294,2989980.729,483448.3473,1298811.962,3106784.5,386289.5525,4875376.339,4791146.877,5996088.922,3013612.505,470541.3290,1093819.181,1,5,25
1,2017-02-01,0.076213,0.316156,113438.0949,16187.0,1590979.0,4659.0,11528.0,272961.0,1318018.0,1025704.960,1007968.299,228359.839,530826.1625,199731.6950,49050.6025,0.0678,14753.34784,206242.568,5.18,2881.68,53.40,54.01,102.919033,-24.3,-19.9,-31.0,10.504292,1.574855e+11,8.351517e+10,1.096908e+11,8.252536e+10,8.582517e+10,1.522091e+11,5.056706e+10,6.034851e+10,2.138243e+10,2.216364e+10,2.104634e+11,1.036176e+12,...,6566090.070,7930536.312,4347164.930,677598.1836,1864166.572,4353892.017,433679.7772,5854575.608,4445890.861,6402426.269,3390803.016,550883.0739,1302564.56,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.000,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,3814229.013,632387.5921,1627959.289,4068020.9,438682.2679,6513619.182,6137144.374,6675101.294,2989980.729,483448.3473,1298811.962,0,4,24


## Dating

What happends on Monday has some influence on what happens onb Tuesday.  Even if we are not going to scale the date column we cannot process the data set as is .  Weneed to change the format tof  the data column. W There are a few ways of doing this including converting them into arbtriry neumbers.  We chose to convert our dates to ORDIANL.  this maintains some relation between the numbers.  In the future we will explore Face boodk's Prophet appraoch to dates.

In [60]:
data['date'] = pd.to_datetime(data['date'])
data['date']= data['date'].map(dt.datetime.toordinal)

Later we will need this list of Feature names 

In [61]:
feature_list = data.loc[: ,data.columns != 'Quantity']
features = list(feature_list)
data.head(3)

# Notice the date column

,date,Quantity,Seasonal,Consumer_loans,Construction_lic_unt,Construction_lic_area,Construction_lic_unt_SI,Construction_lic_unt_NO_SI,Construction_lic_area_SI,Construction_lic_area_NO_SI,Gray_cement_prod,Gray_cement_dispatch,Gray_cement_dispatch_Factory,Gray_cement_dispatch_comercial,Gray_cement_dispatch_contractor,Gray_cement_dispatch_other,DTF(interest_rate),GDP_Constr,GDP_Total,Inflation,USD_exchange,Oil-brent,Oil_WTI,ISE,ICC,IEC,ICE,Unemployment%,RADAR_Constr,RADAR_Rev,RADAR_Toilets,RADAR_HomeAppliance,RADAR_Paint,RADAR_Furniture,RADAR_Tools,RADAR_Elect,RADAR_Plumbing,RADAR_Wood,RADAR_Supply,RADAR_Remodel,...,Preferencial_creditlag_8,loan_overdraftslag_8,Credit_cardlag_8,Treasury_loanlag_8,Housing_loanlag_8,Consumer_creditlag_9,Consumer_microcreditlag_9,Ordinary_creditlag_9,Preferencial_creditlag_9,loan_overdraftslag_9,Credit_cardlag_9,Treasury_loanlag_9,Housing_loanlag_9,Consumer_creditlag_10,Consumer_microcreditlag_10,Ordinary_creditlag_10,Preferencial_creditlag_10,loan_overdraftslag_10,Credit_cardlag_10,Treasury_loanlag_10,Housing_loanlag_10,Consumer_creditlag_11,Consumer_microcreditlag_11,Ordinary_creditlag_11,Preferencial_creditlag_11,loan_overdraftslag_11,Credit_cardlag_11,Treasury_loanlag_11,Housing_loanlag_11,Consumer_creditlag_12,Consumer_microcreditlag_12,Ordinary_creditlag_12,Preferencial_creditlag_12,loan_overdraftslag_12,Credit_cardlag_12,Treasury_loanlag_12,Housing_loanlag_12,holiday,sunday,working_day
0,736330,1.503548,0.699923,112579.4297,11303.0,1185131.0,3587.0,7716.0,229478.0,955653.0,897063.518,913190.407,194276.0200,502084.8330,171559.9665,45269.5875,0.0694,14753.34784,206242.568,5.47,2944.65,52.19,52.81,102.548972,-30.2,-28.3,-33.1,11.733267,1.044525e+11,1.323489e+11,1.727682e+11,1.050914e+11,8.847567e+10,8.354725e+10,5.008444e+10,9.372000e+10,3.037115e+10,3.136015e+10,1.585430e+11,1.050763e+12,...,4445890.861,6402426.269,3390803.016,550883.0739,1302564.560,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.000,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,3814229.013,632387.5921,1627959.289,4068020.900,438682.2679,6513619.182,6137144.374,6675101.294,2989980.729,483448.3473,1298811.962,3106784.500,386289.5525,4875376.339,4791146.877,5996088.922,3013612.505,470541.3290,1093819.181,1,5,25
1,736361,0.076213,0.316156,113438.0949,16187.0,1590979.0,4659.0,11528.0,272961.0,1318018.0,1025704.960,1007968.299,228359.8390,530826.1625,199731.6950,49050.6025,0.0678,14753.34784,206242.568,5.18,2881.68,53.40,54.01,102.919033,-24.3,-19.9,-31.0,10.504292,1.574855e+11,8.351517e+10,1.096908e+11,8.252536e+10,8.582517e+10,1.522091e+11,5.056706e+10,6.034851e+10,2.138243e+10,2.216364e+10,2.104634e+11,1.036176e+12,...,6566090.070,7930536.312,4347164.930,677598.1836,1864166.572,4353892.017,433679.7772,5854575.608,4445890.861,6402426.269,3390803.016,550883.0739,1302564.560,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.000,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,3814229.013,632387.5921,1627959.289,4068020.900,438682.2679,6513619.182,6137144.374,6675101.294,2989980.729,483448.3473,1298811.962,0,4,24
2,736389,-1.346258,-0.267695,114667.2587,15231.0,1438528.0,5687.0,9544.0,354904.0,1083624.0,1099919.669,1083268.231,241130.4555,568447.4880,218640.3600,55049.9274,0.0665,14753.34784,206242.568,4.69,2943.49,50.43,50.60,102.162670,-21.1,-18.3,-25.4,9.704610,1.215372e+11,9.821204e+10,1.191497e+11,9.035507e+10,9.516119e+10,1.388751e+11,5.266968e+10,1.137228e+11,2.872364e+10,2.258877e+10,1.192185e+11,1.000214e+12,...,4119152.514,6258157.459,3447128.397,478975.7922,1302860.802,5430241.566,555441.9044,7916686.911,6566090.070,7930536.312,4347164.930,677598.1836,1864166.572,4353892.017,433679.7772,5854575.608,4445890.861,6402426.269,3390803.016,550883.0739,1302564.560,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.000,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,381

Last round we peeled off  the data column , scaled the data, then re-attached the date column.  Might have guessed that Python had an esier way to approach this.

https://stackoverflow.com/questions/62997793/apply-standarization-to-all-columns-except-one-in-pandas

In [81]:
# except B, take all columns
#cols = df.columns.difference(['B'])
#
cols =data.columns.difference(['date'])

In [82]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [83]:
# normalise only selected columns
#df[cols] = sc.fit_transform(df[cols])
#
data[cols] = sc.fit_transform(data[cols])

The date column is Ordial while the reso of te 943 Features are scaled  

In [84]:
data.head(3)

,date,Quantity,Seasonal,Consumer_loans,Construction_lic_unt,Construction_lic_area,Construction_lic_unt_SI,Construction_lic_unt_NO_SI,Construction_lic_area_SI,Construction_lic_area_NO_SI,Gray_cement_prod,Gray_cement_dispatch,Gray_cement_dispatch_Factory,Gray_cement_dispatch_comercial,Gray_cement_dispatch_contractor,Gray_cement_dispatch_other,DTF(interest_rate),GDP_Constr,GDP_Total,Inflation,USD_exchange,Oil-brent,Oil_WTI,ISE,ICC,IEC,ICE,Unemployment%,RADAR_Constr,RADAR_Rev,RADAR_Toilets,RADAR_HomeAppliance,RADAR_Paint,RADAR_Furniture,RADAR_Tools,RADAR_Elect,RADAR_Plumbing,RADAR_Wood,RADAR_Supply,RADAR_Remodel,...,Preferencial_creditlag_8,loan_overdraftslag_8,Credit_cardlag_8,Treasury_loanlag_8,Housing_loanlag_8,Consumer_creditlag_9,Consumer_microcreditlag_9,Ordinary_creditlag_9,Preferencial_creditlag_9,loan_overdraftslag_9,Credit_cardlag_9,Treasury_loanlag_9,Housing_loanlag_9,Consumer_creditlag_10,Consumer_microcreditlag_10,Ordinary_creditlag_10,Preferencial_creditlag_10,loan_overdraftslag_10,Credit_cardlag_10,Treasury_loanlag_10,Housing_loanlag_10,Consumer_creditlag_11,Consumer_microcreditlag_11,Ordinary_creditlag_11,Preferencial_creditlag_11,loan_overdraftslag_11,Credit_cardlag_11,Treasury_loanlag_11,Housing_loanlag_11,Consumer_creditlag_12,Consumer_microcreditlag_12,Ordinary_creditlag_12,Preferencial_creditlag_12,loan_overdraftslag_12,Credit_cardlag_12,Treasury_loanlag_12,Housing_loanlag_12,holiday,sunday,working_day
0,736330,1.539880,0.703768,-1.632022,-0.889269,-0.528241,-1.103651,-0.287798,-1.101781,0.016399,-0.966155,-0.621275,-0.553653,-0.536381,-0.598653,-0.844779,1.901225,0.728906,-0.754819,2.388963,-0.944942,-0.457605,-0.111201,-0.687592,-1.480974,-2.156877,-0.465022,-0.035719,-0.466215,0.433547,2.497895,-0.991226,-1.232210,-1.831242,-0.868114,-0.447471,0.574186,1.565983,-0.307755,-0.863563,...,-1.053966,0.078346,-1.394121,1.266696,-1.108110,-0.624929,-0.245088,-0.776366,-0.902077,0.527733,-1.466087,1.210525,-0.725630,-0.460762,0.388959,0.732856,0.044353,1.414693,-0.772702,1.753148,-0.299252,-0.934209,-0.431109,-0.055554,0.038088,0.145483,-1.766711,0.619969,-1.080612,-1.414953,-0.862847,-1.618821,-0.829065,-0.536741,-1.656385,0.501004,-1.506361,-0.394158,1.357242,0.380225
1,736361,0.110421,0.325418,-1.579833,0.226266,0.580418,-0.737166,1.359021,-0.859717,1.388236,-0.090003,0.066074,0.352555,-0.137113,0.353344,-0.470100,1.784625,0.728906,-0.754819,2.073044,-1.130383,-0.363636,-0.007877,-0.608165,-0.985189,-1.363913,-0.365731,-0.403188,1.136662,-1.261025,-0.744487,-1.477211,-1.406964,-0.376515,-0.829790,-1.576824,-0.897072,0.109178,0.761352,-0.959579,...,0.292836,1.272364,-0.174010,2.245952,0.223510,-0.800223,-0.470325,-0.634713,-1.035486,0.026740,-1.331254,1.218904,-1.084725,-0.602152,-0.224741,-0.774801,-0.892580,0.473138,-1.460572,1.138999,-0.709756,-0.461524,0.379834,0.724051,0.027079,1.487196,-0.727822,1.731090,-0.300246,-0.903358,-0.409989,-0.045720,0.038952,0.117254,-1.685606,0.597652,-1.030819,-0.653725,-0.736788,0.120324
2,736389,-1.314169,-0.250193,-1.505126,0.007910,0.163966,-0.385724,0.501915,-0.403552,0.500870,0.415460,0.612165,0.692095,0.385513,0.992317,0.124403,1.689888,0.728906,-0.754819,1.539249,-0.948358,-0.594287,-0.301490,-0.770505,-0.716289,-1.212872,-0.100955,-0.642296,0.050155,-0.751032,-0.258268,-1.308589,-0.791415,-0.659020,-0.662823,0.229459,0.304524,0.176521,-1.117497,-1.196299,...,-1.261519,-0.034381,-1.322262,0.710995,-1.107407,-0.222414,0.547229,1.285348,0.302120,1.282045,-0.130459,2.189370,0.241106,-0.778673,-0.456993,-0.632586,-1.028030,-0.035971,-1.321413,1.147133,-1.070086,-0.602985,-0.240056,-0.844457,-0.916897,0.454280,-1.396862,1.114885,-0.711088,-0.438214,0.395311,0.724021,0.027951,1.528374,-0.666388,1.712914,-0.267264,-0.394158,-0.736788,0.640126


## Split the data into Training and Testing

y is our dependent variable/Feature

In [85]:
y = data['Quantity']

 X is every other Feature EXCEPT Quantity

In [86]:
X = data.loc[: ,data.columns != 'Quantity']

In [87]:
model = Lasso()

In [88]:
model.fit(X_train, y_train)

Lasso()

In [89]:
print("Accuracy score for training data", model.score(X_train,y_train))

Accuracy score for training data 0.06614793513532946


In [90]:
print("Accuracy score for test Data", model.score(X_test,y_test))

Accuracy score for test Data 0.043636967663260195


## Grid Search

In [91]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

"search" will iterate Lasso's "Alpha" from 0.1 to 10 in 0.1 incriments

In [92]:
search = GridSearchCV(pipeline, # <<<--- "pipline" = StandardScaler and  Lasso regression
                      {'model__alpha':np.arange(0.1,10,0.1)}, # Alpha setting  
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

Nwo we call Search 

In [93]:
search # Spell out the Alpha setting iterations 0.1 0.2...9.9

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [94]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.607 total time=   0.0s
[CV 2/5] END .................model__alpha=0.1;, score=-0.725 total time=   0.1s
[CV 3/5] END .................model__alpha=0.1;, score=-0.330 total time=   0.1s
[CV 4/5] END .................model__alpha=0.1;, score=-0.655 total time=   0.1s
[CV 5/5] END .................model__alpha=0.1;, score=-0.541 total time=   0.1s
[CV 1/5] END .................model__alpha=0.2;, score=-0.487 total time=   0.0s
[CV 2/5] END .................model__alpha=0.2;, score=-0.634 total time=   0.1s
[CV 3/5] END .................model__alpha=0.2;, score=-0.162 total time=   0.1s
[CV 4/5] END .................model__alpha=0.2;, score=-0.780 total time=   0.1s
[CV 5/5] END .................model__alpha=0.2;, score=-0.433 total time=   0.1s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.440 total time=   0.0s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [71]:
X_train, X_test,y_train,y_test = train_test_split(X , y , test_size= 0.5 , random_state= 1)

In [72]:
model.fit(X_train, y_train)

Lasso()

In [73]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.607 total time=   0.0s
[CV 2/5] END .................model__alpha=0.1;, score=-0.725 total time=   0.1s
[CV 3/5] END .................model__alpha=0.1;, score=-0.330 total time=   0.1s
[CV 4/5] END .................model__alpha=0.1;, score=-0.655 total time=   0.1s
[CV 5/5] END .................model__alpha=0.1;, score=-0.541 total time=   0.1s
[CV 1/5] END .................model__alpha=0.2;, score=-0.487 total time=   0.1s
[CV 2/5] END .................model__alpha=0.2;, score=-0.634 total time=   0.1s
[CV 3/5] END .................model__alpha=0.2;, score=-0.162 total time=   0.1s
[CV 4/5] END .................model__alpha=0.2;, score=-0.780 total time=   0.1s
[CV 5/5] END .................model__alpha=0.2;, score=-0.433 total time=   0.0s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.440 total time=   0.1s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

Lasso's Alpha works by multiplying each Feature value in turn .  the goeal is to get as many Features coefficients to zero as it can . Something x 0 = nothing and that is how Leduces dimesionality .  Counter intuitivly multiplying the Features by less than the bigest number in this case was best

In [95]:
search.best_params_

{'model__alpha': 0.4}

In [96]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [ ]:
coefficients

In [98]:
importance = np.abs(coefficients)

In [99]:
importance

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [100]:
np.array(features)[importance > 0]

array(['RADAR_Plumbinglag_6', 'Housing_total_sales_SIlag_20'],
      dtype='<U37')

In [80]:
np.array(features)[importance != 0]

array(['RADAR_Plumbinglag_6', 'Housing_total_sales_SIlag_20'],
      dtype='<U37')

side by side

In [101]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [102]:
data= pd.read_csv('/content/drive/MyDrive/Porcelana_OP_adelphi_2021_SCALED.csv')
data.head(2)

,date,Quantity,Seasonal,Consumer_loans,Construction_lic_unt,Construction_lic_area,Construction_lic_unt_SI,Construction_lic_unt_NO_SI,Construction_lic_area_SI,Construction_lic_area_NO_SI,Gray_cement_prod,Gray_cement_dispatch,Gray_cement_dispatch_Factory,Gray_cement_dispatch_comercial,Gray_cement_dispatch_contractor,Gray_cement_dispatch_other,DTF(interest_rate),GDP_Constr,GDP_Total,Inflation,USD_exchange,Oil-brent,Oil_WTI,ISE,ICC,IEC,ICE,Unemployment%,RADAR_Constr,RADAR_Rev,RADAR_Toilets,RADAR_HomeAppliance,RADAR_Paint,RADAR_Furniture,RADAR_Tools,RADAR_Elect,RADAR_Plumbing,RADAR_Wood,RADAR_Supply,RADAR_Remodel,...,Preferencial_creditlag_8,loan_overdraftslag_8,Credit_cardlag_8,Treasury_loanlag_8,Housing_loanlag_8,Consumer_creditlag_9,Consumer_microcreditlag_9,Ordinary_creditlag_9,Preferencial_creditlag_9,loan_overdraftslag_9,Credit_cardlag_9,Treasury_loanlag_9,Housing_loanlag_9,Consumer_creditlag_10,Consumer_microcreditlag_10,Ordinary_creditlag_10,Preferencial_creditlag_10,loan_overdraftslag_10,Credit_cardlag_10,Treasury_loanlag_10,Housing_loanlag_10,Consumer_creditlag_11,Consumer_microcreditlag_11,Ordinary_creditlag_11,Preferencial_creditlag_11,loan_overdraftslag_11,Credit_cardlag_11,Treasury_loanlag_11,Housing_loanlag_11,Consumer_creditlag_12,Consumer_microcreditlag_12,Ordinary_creditlag_12,Preferencial_creditlag_12,loan_overdraftslag_12,Credit_cardlag_12,Treasury_loanlag_12,Housing_loanlag_12,holiday,sunday,working_day
0,2017-01-01,1.503548,0.699923,112579.4297,11303.0,1185131.0,3587.0,7716.0,229478.0,955653.0,897063.518,913190.407,194276.020,502084.8330,171559.9665,45269.5875,0.0694,14753.34784,206242.568,5.47,2944.65,52.19,52.81,102.548972,-30.2,-28.3,-33.1,11.733267,1.044525e+11,1.323489e+11,1.727682e+11,1.050914e+11,8.847567e+10,8.354725e+10,5.008444e+10,9.372000e+10,3.037115e+10,3.136015e+10,1.585430e+11,1.050763e+12,...,4445890.861,6402426.269,3390803.016,550883.0739,1302564.560,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.00,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,3814229.013,632387.5921,1627959.289,4068020.900,438682.2679,6513619.182,6137144.374,6675101.294,2989980.729,483448.3473,1298811.962,3106784.5,386289.5525,4875376.339,4791146.877,5996088.922,3013612.505,470541.3290,1093819.181,1,5,25
1,2017-02-01,0.076213,0.316156,113438.0949,16187.0,1590979.0,4659.0,11528.0,272961.0,1318018.0,1025704.960,1007968.299,228359.839,530826.1625,199731.6950,49050.6025,0.0678,14753.34784,206242.568,5.18,2881.68,53.40,54.01,102.919033,-24.3,-19.9,-31.0,10.504292,1.574855e+11,8.351517e+10,1.096908e+11,8.252536e+10,8.582517e+10,1.522091e+11,5.056706e+10,6.034851e+10,2.138243e+10,2.216364e+10,2.104634e+11,1.036176e+12,...,6566090.070,7930536.312,4347164.930,677598.1836,1864166.572,4353892.017,433679.7772,5854575.608,4445890.861,6402426.269,3390803.016,550883.0739,1302564.56,4680431.000,460632.0000,5702443.000,4657354.000,7012296.000,3283417.000,549789.0000,1454672.000,4941981.754,531850.1926,7315235.649,6120086.089,8140200.383,3814229.013,632387.5921,1627959.289,4068020.9,438682.2679,6513619.182,6137144.374,6675101.294,2989980.729,483448.3473,1298811.962,0,4,24


In [103]:
data['date'] = pd.to_datetime(data['date'])
data['date']= data['date'].map(dt.datetime.toordinal)

In [104]:
#y = data_robust_scaled_named['diagnosis']
y = data['Quantity']

In [105]:
X = data.loc[: ,data.columns != 'Quantity']

In [106]:
#X = data.loc[: ,data.columns != 'Quantity']
feature_list = data.loc[: ,data.columns != 'Quantity']
#features = list(data)

In [107]:
features = list(feature_list)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [109]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [110]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [111]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.483 total time=   0.1s
[CV 2/5] END .................model__alpha=0.1;, score=-0.723 total time=   0.1s
[CV 3/5] END .................model__alpha=0.1;, score=-1.598 total time=   0.1s
[CV 4/5] END .................model__alpha=0.1;, score=-0.668 total time=   0.1s
[CV 5/5] END .................model__alpha=0.1;, score=-0.729 total time=   0.1s
[CV 1/5] END .................model__alpha=0.2;, score=-0.383 total time=   0.1s
[CV 2/5] END .................model__alpha=0.2;, score=-0.774 total time=   0.1s
[CV 3/5] END .................model__alpha=0.2;, score=-1.796 total time=   0.1s
[CV 4/5] END .................model__alpha=0.2;, score=-0.724 total time=   0.1s
[CV 5/5] END .................model__alpha=0.2;, score=-0.478 total time=   0.1s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.432 total time=   0.1s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [112]:
search.best_params_

{'model__alpha': 0.2}

In [113]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [114]:
importance = np.abs(coefficients)

In [ ]:
#importance

In [115]:
np.array(features)[importance > 0]

array(['Seasonal', 'Gray_cement_dispatch_comercial', 'ISE',
       'Construction_lic_area/unt_SIlag_4', 'IEClag_5', 'ISElag_6',
       'Gray_cement_dispatch_Factorylag_9', 'ICElag_9',
       'Housing_total_sales', 'Housing_total_initiation_SIlag_24',
       'Ordinary_creditlag_4', 'Ordinary_creditlag_6', 'working_day'],
      dtype='<U37')

In [118]:
Selected_Features = ['Seasonal', 'Gray_cement_dispatch_comercial', 'ISE',
       'Construction_lic_area/unt_SIlag_4', 'IEClag_5', 'ISElag_6',
       'Gray_cement_dispatch_Factorylag_9', 'ICElag_9',
       'Housing_total_sales', 'Housing_total_initiation_SIlag_24',
       'Ordinary_creditlag_4', 'Ordinary_creditlag_6', 'working_day']

In [119]:
Selected_Features

['Seasonal',
 'Gray_cement_dispatch_comercial',
 'ISE',
 'Construction_lic_area/unt_SIlag_4',
 'IEClag_5',
 'ISElag_6',
 'Gray_cement_dispatch_Factorylag_9',
 'ICElag_9',
 'Housing_total_sales',
 'Housing_total_initiation_SIlag_24',
 'Ordinary_creditlag_4',
 'Ordinary_creditlag_6',
 'working_day']